In [ ]:
#Trend 1: Players age 20-24 spend the most Pymoli Cents. They don't buy significantly more expensive items, but rather
#the quantity of items purchases by this group is much greater than other age categories. Likewise, men spend more. Men
#also make up a large percentage of the player base, so this is hardly surprising. Again, the amount per item spent by 
#is similar to the amount per item spent by women. 

#Trend 2: Top spending players indicate more revenue can be captured outside the 20-24 male demographic. Though 2 out 
#of 5 of the top spending players fall into this demographic, 3 do not. Mindimnya67 is female player, age 39 who has 
#has spent 12.74 Pymoli Cents. Sondim43 and Saedue76 are both male but they are 20 and 25 respectively. Together, they 
#have spent 26.58 Pymoli Cents.

#Trend 3: Cheaper items sell better, increasing purchase of Pymoli Cents. Though some of our higher priced items sell 
#well, most of the profitable items are low-to medium priced, and in demand among the player base. The highest priced
#best seller is the Spectral Diamond Doomblade, at a mere 4 and a quarter Pymoli cent cost. 

In [3]:
# Initial Setup. Note: I joined both datasets into a single one, but from what I can tell the
#script should work on each individually as well.
import pandas as pd
import numpy as np
heroesPymoli_df1 = pd.read_json("purchase_data.json")
heroesPymoli_df2 = pd.read_json("purchase_data2.json")
frames = [heroesPymoli_df1,heroesPymoli_df2]
heroesPymoli_df = pd.concat(frames)
heroesPymoli_df = heroesPymoli_df.rename(columns = {"Price":"Price in Pymoli Cents"})

#Retrieve number of players
screen_names = heroesPymoli_df["SN"].unique()
screen_names.size
heroesPymoli_df.head()

,Age,Gender,Item ID,Item Name,Price in Pymoli Cents,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [4]:
#Purchasing Analysis, I jokingly created an in-game currency as the value (I assume players can buy with
#real currency in the country they live in)
game_items = heroesPymoli_df["Item Name"].unique()
game_items.size
heroesPymoli_df = heroesPymoli_df[["SN", "Age", "Gender", "Item ID", "Item Name", "Price in Pymoli Cents"]]
average_purchase = (heroesPymoli_df["Price in Pymoli Cents"].sum())/(screen_names.size)
average_purchase
purchases = len(heroesPymoli_df.index)
purchases
total_revenue = heroesPymoli_df["Price in Pymoli Cents"].sum()
total_revenue

d = {"Unique Game Items": (game_items.size), 
     "Average Pymoli Cents Spent" : (average_purchase), 
     "Number Purchases" : (purchases),
    "Total Revenue" : (total_revenue)}
purchasing_df = pd.DataFrame(data = d, index=[0])
purchasing_df

,Average Pymoli Cents Spent,Number Purchases,Total Revenue,Unique Game Items
0,4.108546,858,2514.43,180


In [5]:
#Gender Demographics
heroesPymoli_players = heroesPymoli_df.drop_duplicates(["SN"])
females = heroesPymoli_players.loc[heroesPymoli_players.Gender == "Female", "Gender"].count()
males = heroesPymoli_players.loc[heroesPymoli_players.Gender == "Male", "Gender"].count()
other = screen_names.size - (females + males)
genders = ["Female", "Male", "Other/Not Disclosed"]
gender_data = {"Total Count" : [females, males, other]}
gender_demo = pd.DataFrame(data = gender_data, index = genders)
percent_players = lambda x: (x/screen_names.size)*100
gender_demo["Percentage of Players"] = (gender_demo["Total Count"].map(percent_players)).round(2)
gender_demo                                     

,Total Count,Percentage of Players
Female,108,17.65
Male,495,80.88
Other/Not Disclosed,9,1.47


In [6]:
#Gender Payment Analysis
#First few columns...
gender_group = heroesPymoli_df.groupby( ["Gender"] )
gender_purchases = gender_group.size().to_frame(name = "Purchases").reset_index()
total_purchase = gender_group.aggregate(np.sum)
total_purchase = total_purchase.set_index(gender_purchases.index)
gender_purchases["Total Purchase Value"] = (total_purchase["Price in Pymoli Cents"])
gender_demo = gender_demo.set_index(gender_purchases.index)
gender_purchase_avg = (total_purchase["Price in Pymoli Cents"].divide(gender_demo["Total Count"]))
gender_purchases["Average Purchase Value"] = gender_purchase_avg


#Additional Calculation for "Normalized" values. 
#Used this method: https://en.wikipedia.org/wiki/Feature_scaling#Rescaling
min_price = heroesPymoli_df["Price in Pymoli Cents"].min()
max_price = heroesPymoli_df["Price in Pymoli Cents"].max()
normal_prices = (heroesPymoli_df["Price in Pymoli Cents"]).map(lambda x: (x - min_price) / (max_price - min_price))
heroesPymoli_df["Normalized Prices"] = normal_prices
gender_group_norm = heroesPymoli_df.groupby( ["Gender"] )
total_purchase_norm = gender_group_norm.aggregate(np.sum)
total_purchase_norm = total_purchase_norm.set_index(gender_purchases.index)
gender_purchases["Normalized Total"] = total_purchase_norm["Normalized Prices"]
gender_purchases

,Gender,Purchases,Total Purchase Value,Average Purchase Value,Normalized Total
0,Female,149,424.29,3.928611,69.290076
1,Male,697,2052.28,4.146020,341.307888
2,Other / Non-Disclosed,12,37.86,4.206667,6.519084


In [7]:
#Age Demographics
bins = [0, 10, 15, 20, 25, 30, 35, 40,
        (heroesPymoli_df["Age"].max())]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
heroesPymoli_df["Age Group"] = pd.cut(heroesPymoli_df["Age"], bins, labels=age_labels)
age_group = heroesPymoli_df.groupby("Age Group")

age_purchases = age_group.size().to_frame(name = "Purchases").reset_index()
total_purchase = age_group.aggregate(np.sum)
total_purchase = total_purchase.set_index(age_purchases.index)
age_purchases["Total Purchase Value"] = (total_purchase["Price in Pymoli Cents"])
age_purchases["Average Purchase Value"] = (age_purchases["Total Purchase Value"].divide(age_purchases["Purchases"]))

age_group_norm = heroesPymoli_df.groupby( ["Age"] )
total_purchase_norm = age_group_norm.aggregate(np.sum)
age_purchases["Normalized Total"] = (total_purchase["Normalized Prices"])
age_purchases


,Age Group,Purchases,Total Purchase Value,Average Purchase Value,Normalized Total
0,<10,37,110.44,2.984865,18.498728
1,10-14,82,236.36,2.882439,38.860051
2,15-19,204,583.43,2.859951,95.508906
3,20-24,338,1003.03,2.967544,167.498728
4,25-29,80,230.59,2.882375,37.910941
5,30-34,65,194.73,2.995846,32.679389
6,35-39,49,147.21,3.004286,24.740458
7,40+,3,8.64,2.880000,1.419847


In [8]:
#Top Spending Players
player_group = heroesPymoli_df.groupby(["SN"])
player_purchases = player_group.size().to_frame(name = "Purchases").reset_index()
total_purchase = player_group.aggregate(np.sum)
total_purchase = total_purchase.set_index(player_purchases.index)
player_purchases["Total Purchase Value"] = (total_purchase["Price in Pymoli Cents"])
player_purchases = player_purchases.sort_values("Total Purchase Value", ascending = False)
player_purchases = player_purchases.reset_index(drop=True)
player_purchases["Average Purchase Value"] = (player_purchases["Total Purchase Value"].divide(player_purchases["Purchases"]))
player_purchases.head()

,SN,Purchases,Total Purchase Value,Average Purchase Value
0,Undirrala66,5,17.06,3.412
1,Aerithllora36,4,15.10,3.775
2,Saedue76,4,13.56,3.390
3,Sondim43,4,13.02,3.255
4,Mindimnya67,4,12.74,3.185


In [13]:
#Popular Items
id_lookup = heroesPymoli_df.set_index("Item Name").to_dict()["Item ID"]
item_group = heroesPymoli_df.groupby(["Item Name"])
item_purchases = item_group.size().to_frame(name = "Purchases").reset_index()
total_purchase = item_group.aggregate(np.sum)
total_purchase = total_purchase.set_index(item_purchases.index)
item_purchases["Total Purchase Value"] = (total_purchase["Price in Pymoli Cents"])
item_purchases = item_purchases.sort_values("Purchases", ascending = False)
item_purchases["Item ID"] = item_purchases["Item Name"].map(id_lookup)
item_purchases.head()

,Item Name,Purchases,Total Purchase Value,Item ID
56,Final Critic,14,38.60,101
8,Arcane Gem,12,29.34,84
138,Stormcaller,12,40.19,180
11,"Betrayal, Whisper of Grieving Widows",11,25.85,39
156,Trickster,10,23.22,31


In [14]:
#Profitable Items
item_purchases.sort_values("Total Purchase Value", ascending = False).head()

,Item Name,Purchases,Total Purchase Value,Item ID
138,Stormcaller,12,40.19,180
56,Final Critic,14,38.60,101
113,Retribution Axe,9,37.26,34
135,"Splitter, Foe Of Subtlety",9,33.03,107
133,Spectral Diamond Doomblade,7,29.75,115
